# Keras Implementation - Human Data Analytics Project 

Importing all modules that we need for our elaboration

In [1]:
import numpy as np
import scipy.io

from keras import regularizers

import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=FutureWarning)
    
    from keras.layers import Conv1D, BatchNormalization, Dropout, LeakyReLU, Flatten, Activation, Dense, MaxPooling1D
    from keras.models import Model, Sequential
    from keras.optimizers import Adam
    from sklearn.preprocessing import OneHotEncoder, StandardScaler
    from sklearn.metrics import f1_score, roc_curve, auc
    import keras.backend as K
    
K.set_image_data_format('channels_last')

%matplotlib inline

C:\Users\Matteo\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Preprocessing Section

In [2]:
x1 = scipy.io.loadmat("data_temp/S1-ADL1", mdict={'filled_features':'features', 'labels':'labels'})
x2 = scipy.io.loadmat("data_temp/S1-ADL2", mdict={'filled_features':'features', 'labels':'labels'})
x3 = scipy.io.loadmat("data_temp/S1-ADL3", mdict={'filled_features':'features', 'labels':'labels'})
x4 = scipy.io.loadmat("data_temp/S1-ADL4", mdict={'filled_features':'features', 'labels':'labels'})
x5 = scipy.io.loadmat("data_temp/S1-ADL5", mdict={'filled_features':'features', 'labels':'labels'})

We split the data into train and test set.

In [3]:
X_train = np.concatenate((x1['filled_features'],x2['filled_features'],x3['filled_features']),axis=0)
X_test = np.concatenate((x4['filled_features'],x5['filled_features']),axis=0)
#X_test = x4['filled_features']

Y_train = np.concatenate((x1['labels'][:,0],x2['labels'][:,0],x3['labels'][:,0]),axis=0) # here we take just the first column because we want to fulfill the first task
#Y_test = x4['labels'][:,0]
Y_test = np.concatenate((x4['labels'][:,0],x5['labels'][:,0]),axis=0)

# Initial decision to overcome the problem of entire missing columns
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

print("X_train shape: ", X_train.shape)
print("X_test shape: ", X_test.shape)


X_train shape:  (104973, 113)
X_test shape:  (57536, 113)


The dataset needs to be normalized in order to improve performances.

In [4]:
# fatures normalization
scaler = StandardScaler().fit(X_train)
X_train =scaler.transform(X_train)
X_test = scaler.transform(X_test)

We decided to use a onehot encoder for the labels

In [5]:
onehot_encoder = OneHotEncoder(sparse=False)
YOH_train = onehot_encoder.fit_transform(Y_train.reshape(-1, 1))
YOH_test = onehot_encoder.fit_transform(Y_test.reshape(-1, 1))

print("YOH_train shape: ", YOH_train.shape)
print("YOH_test shape: ", YOH_test.shape)

YOH_train shape:  (104973, 5)
YOH_test shape:  (57536, 5)


In [6]:
window_size = 10
stride = int(window_size / 2)
num_features = X_train.shape[1]
samples, classes = YOH_train.shape
print("Number of samples: ", samples, "\nNumber of classes: ", classes)

windows = int(samples // stride) - 1
print("Number of windows: ", windows)

Number of samples:  104973 
Number of classes:  5
Number of windows:  20993


In [7]:
num_features_test = X_test.shape[1]
samples_test, classes_test = YOH_test.shape
print("Number of samples: ", samples_test, "\nNumber of classes: ", classes_test)

windows_test = int(samples_test // stride) - 1
print("Number of windows: ", windows_test)

Number of samples:  57536 
Number of classes:  5
Number of windows:  11506


In [8]:
cube = np.zeros([windows, window_size, num_features])
cube.shape

(20993, 10, 113)

In [9]:
cube_test = np.zeros([windows_test, window_size, num_features])
cube_test.shape

(11506, 10, 113)

In [10]:
lab_cum = np.zeros([windows])
lab_cum.shape

(20993,)

In [11]:
lab_cum_test = np.zeros([windows_test])
lab_cum_test.shape

(11506,)

In [12]:
for w in range(windows):
    index = int(w * stride)
    cube[w,:,:] = X_train[index:index+window_size, :].reshape((window_size,num_features))
    l = YOH_train[index:index+window_size,:] # shape 50 x 5
    #print(np.sum(l, axis=0),np.argmax(np.sum(l, axis=0)))
    lab_cum[w] = np.argmax(np.sum(l, axis=0))

In [13]:
for w in range(windows_test):
    index = int(w * stride)
    cube_test[w,:,:] = X_test[index:index+window_size, :].reshape((window_size,num_features))
    l = YOH_test[index:index+window_size,:] # shape 50 x 5
    #print(np.sum(l, axis=0),np.argmax(np.sum(l, axis=0)))
    lab_cum_test[w] = np.argmax(np.sum(l, axis=0))

In [14]:
#print('cumulative labels: ', lab_cum.shape, type(lab_cum), "\n", lab_cum)
YOH_train_final = onehot_encoder.fit_transform(lab_cum.reshape(-1, 1))
print(YOH_train_final)

[[0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]]


In [15]:
YOH_test_final = onehot_encoder.fit_transform(lab_cum_test.reshape(-1, 1))
print(YOH_test_final)

[[0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]]


In [16]:
def TestModel(input_shape):
    """ 
    Arguments:
    input_shape -- shape of the images of the dataset

    Returns: 
    model -- a Model() instance in Keras
    """
    
    model = Sequential()
    model.add(Conv1D(filters = 18,
                    kernel_size=5,
                    strides=1,
                    padding='same',
                    input_shape = input_shape))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=2,
                          strides=2,
                          padding='same'))
    
    model.add(Conv1D(filters = 36,
                    kernel_size=7,
                    strides=1,
                    padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=2,
                          strides=2,
                          padding='same'))
    
    model.add(Dropout(0.2))
    
    model.add(Conv1D(filters = 72,
                    kernel_size=7,
                    strides=1,
                    padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=2,
                          strides=2,
                          padding='same'))
    
    #model.add(Conv1D(filters = 144,
    #                kernel_size=7,
    #                strides=1,
    #                padding='same'))
    #model.add(BatchNormalization())
    #model.add(Activation('relu'))
    #model.add(MaxPooling1D(pool_size=2,
    #                      strides=2,
    #                      padding='same'))
    
    model.add(Flatten())
    
    model.add(Dense(64, kernel_regularizer=regularizers.l2(0.01)))
    model.add(Activation('relu'))
    
    model.add(Dropout(0.4))

    model.add(Dense(5))
    model.add(Activation('softmax'))
    
    #model.summary()
    
    return model

In [17]:
model_test = TestModel((10,113))

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


In [18]:
opt = Adam(lr=0.01)
model_test.compile(optimizer = opt, loss = "categorical_crossentropy", metrics = ["accuracy"])

In [19]:
model_test.fit(x = cube, y = YOH_train_final, epochs = 50, batch_size = 128, validation_data=(cube_test,YOH_test_final))

Train on 20993 samples, validate on 11506 samples
Epoch 1/50
20993/20993 [==============================] - 5s 238us/step - loss: 0.6129 - acc: 0.8266 - val_loss: 1.6657 - val_acc: 0.7125
Epoch 2/50
20993/20993 [==============================] - 4s 199us/step - loss: 0.4005 - acc: 0.8879 - val_loss: 2.0277 - val_acc: 0.6776
Epoch 3/50
20993/20993 [==============================] - 4s 198us/step - loss: 0.4138 - acc: 0.8989 - val_loss: 2.6377 - val_acc: 0.6609
Epoch 4/50
20993/20993 [==============================] - 4s 199us/step - loss: 0.3566 - acc: 0.9093 - val_loss: 1.2868 - val_acc: 0.7192
Epoch 5/50
20993/20993 [==============================] - 4s 197us/step - loss: 0.5480 - acc: 0.8952 - val_loss: 1.3898 - val_acc: 0.7316
Epoch 6/50
20993/20993 [==============================] - 4s 199us/step - loss: 0.3766 - acc: 0.9129 - val_loss: 1.2419 - val_acc: 0.7342
Epoch 7/50
20993/20993 [==============================] - 4s 199us/step - loss: 0.3659 - acc: 0.9113 - val_loss: 0.9292 - 

In [20]:
y_pred = model_test.predict(cube_test)
print(y_pred)

y_pred_hard = np.argmax(y_pred, axis=1)
print(y_pred_hard)

YOH_pred = onehot_encoder.fit_transform(y_pred_hard.reshape(-1, 1))
print(YOH_pred)

print(f1_score(YOH_test_final, YOH_pred, average='weighted'))

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(classes):
    fpr[i], tpr[i], _ = roc_curve(YOH_test_final[:, i], YOH_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

print(roc_auc)

[[2.9208573e-02 1.3677973e-01 8.3338910e-01 6.0379686e-04 1.8758470e-05]
 [3.4316596e-02 3.0744987e-02 9.3486321e-01 7.0416987e-05 4.7945005e-06]
 [3.1972449e-02 2.8414663e-02 9.3956244e-01 4.7852598e-05 2.5418528e-06]
 ...
 [9.9999964e-01 1.7947408e-09 3.8937472e-07 3.0373761e-26 1.0038408e-15]
 [9.9968278e-01 1.2273900e-05 3.0496059e-04 9.9796056e-16 1.8314679e-10]
 [9.9428272e-01 1.2628589e-03 4.4543939e-03 5.7062870e-09 3.4541152e-08]]
[2 2 2 ... 0 0 0]
[[0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 ...
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]]
0.6877474605643061
{0: 0.8252964481774879, 1: 0.7528821179831148, 2: 0.7719116703078898, 3: 0.8363203695735288, 4: 0.8548162608459118}
